In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

print(tf.__version__)


2.13.0


In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)


In [3]:
train_generator = train_datagen.flow_from_directory(
    "../data/ultrasound/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
    "../data/ultrasound/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

train_generator.class_indices


Found 1924 images belonging to 2 classes.
Found 1922 images belonging to 2 classes.


{'infected': 0, 'notinfected': 1}

In [4]:
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]']   

In [5]:
import os
from PIL import Image

def remove_corrupt_images(base_dir):
    removed = 0
    for root, _, files in os.walk(base_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify image integrity
            except Exception:
                print("Removing corrupt image:", file_path)
                os.remove(file_path)
                removed += 1
    print(f"Total removed images: {removed}")


In [6]:
remove_corrupt_images("../data/ultrasound/train")
remove_corrupt_images("../data/ultrasound/test")


Total removed images: 0
Total removed images: 0


In [7]:
EPOCHS = 15

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)


Epoch 1/15
121/121 [==============================] - 83s 634ms/step - loss: 0.6841 - accuracy: 0.5759 - auc: 0.4873 - val_loss: 0.6786 - val_accuracy: 0.5937 - val_auc: 0.5000
Epoch 2/15
121/121 [==============================] - 153s 1s/step - loss: 0.6805 - accuracy: 0.5852 - auc: 0.5062 - val_loss: 0.6775 - val_accuracy: 0.5937 - val_auc: 0.5000
Epoch 3/15
121/121 [==============================] - 182s 2s/step - loss: 0.6778 - accuracy: 0.5930 - auc: 0.4965 - val_loss: 0.6756 - val_accuracy: 0.5937 - val_auc: 0.5000
Epoch 4/15
121/121 [==============================] - 182s 2s/step - loss: 0.6794 - accuracy: 0.5941 - auc: 0.4802 - val_loss: 0.6752 - val_accuracy: 0.5937 - val_auc: 0.5000
Epoch 5/15
121/121 [==============================] - 186s 2s/step - loss: 0.6774 - accuracy: 0.5951 - auc: 0.5041 - val_loss: 0.6774 - val_accuracy: 0.5937 - val_auc: 0.5000
Epoch 6/15
121/121 [==============================] - 186s 2s/step - loss: 0.6770 - accuracy: 0.5941 - auc: 0.5024 - val_lo

KeyboardInterrupt: 

In [8]:
for layer in base_model.layers[-20:]:
    layer.trainable = True


In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)


In [10]:
FINE_TUNE_EPOCHS = 10

history_fine = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=FINE_TUNE_EPOCHS
)


Epoch 1/10
121/121 [==============================] - 192s 2s/step - loss: 0.6863 - accuracy: 0.5660 - auc: 0.4839 - val_loss: 0.6746 - val_accuracy: 0.5937 - val_auc: 0.5032
Epoch 2/10
121/121 [==============================] - 154s 1s/step - loss: 0.6756 - accuracy: 0.5956 - auc: 0.5296 - val_loss: 0.6755 - val_accuracy: 0.5937 - val_auc: 0.5006
Epoch 3/10
121/121 [==============================] - 83s 687ms/step - loss: 0.6795 - accuracy: 0.5847 - auc: 0.5062 - val_loss: 0.6756 - val_accuracy: 0.5937 - val_auc: 0.8526
Epoch 4/10
121/121 [==============================] - 83s 689ms/step - loss: 0.6761 - accuracy: 0.5842 - auc: 0.5194 - val_loss: 0.6752 - val_accuracy: 0.5937 - val_auc: 0.9155
Epoch 5/10
121/121 [==============================] - 86s 713ms/step - loss: 0.6758 - accuracy: 0.5868 - auc: 0.5262 - val_loss: 0.6724 - val_accuracy: 0.5937 - val_auc: 0.8115
Epoch 6/10
121/121 [==============================] - 86s 712ms/step - loss: 0.6716 - accuracy: 0.5930 - auc: 0.5605 - 

KeyboardInterrupt: 

In [11]:
test_generator.reset()

y_prob = model.predict(test_generator).ravel()
y_true = test_generator.classes

from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

print("Test ROC-AUC:", roc_auc_score(y_true, y_prob))
print(confusion_matrix(y_true, (y_prob >= 0.5).astype(int)))
print(classification_report(y_true, (y_prob >= 0.5).astype(int)))


121/121 [==============================] - 40s 324ms/step
Test ROC-AUC: 0.9779221901402839
[[   0  781]
 [   0 1141]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       781
           1       0.59      1.00      0.75      1141

    accuracy                           0.59      1922
   macro avg       0.30      0.50      0.37      1922
weighted avg       0.35      0.59      0.44      1922



c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

In [12]:
# Invert probabilities so that PCOS = 1
y_prob_fixed = 1 - y_prob

from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

print("Fixed Test ROC-AUC:", roc_auc_score(y_true, y_prob_fixed))

y_pred_fixed = (y_prob_fixed >= 0.5).astype(int)

print(confusion_matrix(y_true, y_pred_fixed))
print(classification_report(y_true, y_pred_fixed))


Fixed Test ROC-AUC: 0.022077809859716024
[[ 781    0]
 [1141    0]]
              precision    recall  f1-score   support

           0       0.41      1.00      0.58       781
           1       0.00      0.00      0.00      1141

    accuracy                           0.41      1922
   macro avg       0.20      0.50      0.29      1922
weighted avg       0.17      0.41      0.23      1922



c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\DELL\miniconda3\envs\pcos\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]